<a href="https://colab.research.google.com/github/Ryan-100/FTL-Capstone-Gr1/blob/Marc/capstoneproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd

health_policy_file = "owid-covid-data.csv"

print("Loading health/policy data...")
health_df = pd.read_csv(health_policy_file)

health_df = health_df[health_df['location'] == 'Japan'].copy()

health_columns = [
    'date',
    'new_cases',
    'new_deaths',
    'hosp_patients',         # Number of hospital patients
    'positive_rate',         # Share of tests that are positive
    'stringency_index'       # Our "policy" metric
]
health_df = health_df[health_columns]


health_df['date'] = pd.to_datetime(health_df['date'])


health_df['new_cases'] = health_df['new_cases'].fillna(0)
health_df['new_deaths'] = health_df['new_deaths'].fillna(0)


health_df['hosp_patients'] = health_df['hosp_patients'].ffill()
health_df['positive_rate'] = health_df['positive_rate'].ffill()
health_df['stringency_index'] = health_df['stringency_index'].ffill()


print("--- Health/Policy Data (First 5 Rows) ---")
print(health_df.head())

Loading health/policy data...
--- Health/Policy Data (First 5 Rows) ---
             date  new_cases  new_deaths  hosp_patients  positive_rate  \
188626 2020-01-05        0.0         0.0            NaN            NaN   
188627 2020-01-06        0.0         0.0            NaN            NaN   
188628 2020-01-07        0.0         0.0            NaN            NaN   
188629 2020-01-08        0.0         0.0            NaN            NaN   
188630 2020-01-09        0.0         0.0            NaN            NaN   

        stringency_index  
188626              0.00  
188627              0.00  
188628              2.78  
188629              2.78  
188630              2.78  


In [7]:

mobility_file = "Global_Mobility_Report.csv"
columns_to_load = [
    'country_region',
    'sub_region_1',
    'date',
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline'
]
print("\nLoading mobility data...")
mobility_df = pd.read_csv(mobility_file,usecols=columns_to_load)



mobility_df = mobility_df[mobility_df['country_region'] == 'Japan'].copy()


mobility_df = mobility_df[mobility_df['sub_region_1'].isnull()]



mobility_columns = [
    'date',
    'retail_and_recreation_percent_change_from_baseline',
    'grocery_and_pharmacy_percent_change_from_baseline',
    'parks_percent_change_from_baseline',
    'transit_stations_percent_change_from_baseline',
    'workplaces_percent_change_from_baseline',
    'residential_percent_change_from_baseline'
]
mobility_df = mobility_df[mobility_columns]


mobility_df['date'] = pd.to_datetime(mobility_df['date'])


mobility_df = mobility_df.ffill()

print("--- Mobility Data (First 5 Rows) ---")
print(mobility_df.head())


Loading mobility data...


/tmp/ipython-input-90464499.py:14: DtypeWarning: Columns (10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  mobility_df = pd.read_csv(mobility_file,usecols=columns_to_load)


--- Mobility Data (First 5 Rows) ---
              date  retail_and_recreation_percent_change_from_baseline  \
5841714 2020-02-15                                               -1.0    
5841715 2020-02-16                                               -9.0    
5841716 2020-02-17                                               -2.0    
5841717 2020-02-18                                                1.0    
5841718 2020-02-19                                                0.0    

         grocery_and_pharmacy_percent_change_from_baseline  \
5841714                                                4.0   
5841715                                               -6.0   
5841716                                                1.0   
5841717                                                2.0   
5841718                                                2.0   

         parks_percent_change_from_baseline  \
5841714                                 7.0   
5841715                               -35.0   
5841

/tmp/ipython-input-90464499.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mobility_df = mobility_df.ffill()


In [8]:
print("\nMerging the two datasets...")

# Merge the two dataframes on the 'date' column
# This puts rows together that have the same date.
merged_df = pd.merge(health_df, mobility_df, on='date', how='inner')

# Set the 'date' as the index. This is best practice for time-series.
merged_df = merged_df.set_index('date')

# Just in case any NaNs slipped through (e.g., at the very start)
merged_df = merged_df.fillna(0)

print("--- Final Merged Data (First 5 Rows) ---")
print(merged_df.head())

print("\nData cleaning is complete! We now have one master dataset.")


Merging the two datasets...
--- Final Merged Data (First 5 Rows) ---
            new_cases  new_deaths  hosp_patients  positive_rate  \
date                                                              
2020-02-15        0.0         0.0            0.0           0.16   
2020-02-16       27.0         1.0            0.0           0.15   
2020-02-17        0.0         0.0            0.0           0.12   
2020-02-18        0.0         0.0            0.0           0.15   
2020-02-19        0.0         0.0            0.0           0.14   

            stringency_index  \
date                           
2020-02-15             19.44   
2020-02-16             19.44   
2020-02-17             19.44   
2020-02-18             19.44   
2020-02-19             19.44   

            retail_and_recreation_percent_change_from_baseline  \
date                                                             
2020-02-15                                               -1.0    
2020-02-16                           

In [ ]:

output_file = "/data/japan_covid_master_data.csv"

merged_df.to_csv(output_file)

print(f"\nSuccessfully saved the clean data to {output_file}")


Successfully saved the clean data to japan_covid_master_data.csv
